# Functions

Functions:
- enable you to break large programs into smaller, simpler pieces
- improve readability and make code more approachable
- allow for reuse and refactoring

## Item 17: Be defensive when iterating over arguments
Sometimes you need to go through the input list more than once.

In [15]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [29]:
def read_visits(data_list):
    for element in data_list:
        yield element

# iterators only produce output once
percentages = normalize(read_visits(visits))
print(percentages)

it = read_visits(visits)
print(list(it))
print(list(it)) # empty list, already exhausted
# no errors will be raised when this happens 
# as it's an expected behaviour

[]
[15, 35, 80]
[]


In [26]:
# Approach 1: exhaust an iterator and keep a copy
def normalize_copy(numbers):
    numbers = list(numbers)
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

percentages = normalize_copy(read_visits(visits))
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [30]:
# Approach 2: return an iterator each time, passing a lambda function
def normalize_func(get_iter):
    total = sum(get_iter())
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result

percentage = normalize_func(lambda: read_visits(visits))
print(percentage)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [14]:
# Approach 3: provide a new container class 
# that implements the iterator protocol (__iter__)

class ReadVisits(object):
    def __init__(self, data):
        self.data = data
        
    def __iter__(self):
        for element in self.data:
            yield element

visits_iterated = ReadVisits(visits)
percentages = normalize(visits_iterated)
print(percentages)

NameError: name 'visits' is not defined

### But how?

```python
def normalize(numbers):
    total = sum(numbers) <- calls ReadVisits.__iter__
    result = []
    for value in numbers: <- calls ReadVisits.__iter__
        percent = 100 * value / total
        result.append(percent)
    return result
```

In [26]:
def normalize_defensive(numbers):
    # a value is an iterator if calling iter on it twice
    # produces the same results (progressing via next)
    if iter(numbers) is iter(numbers): # if it's an iterator
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

normalize_defensive(visits)
normalize_defensive(ReadVisits(visits))
normalize_defensive(iter(visits)) # raises a TypeError

TypeError: Must supply a container

In [35]:
container_1 = ReadVisits(visits)
assert iter(container_1) != iter(container_1)

In [36]:
container_2 = visits
assert iter(container_2) != iter(container_2)

In [39]:
not_container = iter(visits)
assert iter(not_container) == iter(not_container)

## Item 18: Variable positional arguments
Accepting star args (`*args`), optional positional arguments, can make a function call more clear and remove visual noise.

In [37]:
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))

log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are: 1, 2
Hi there


Having to pass an empty list is cumbersome and noisy, would be better to leave out the second argument. Using the `*` prefix on `values` makes them optional.

The `*values` returns a tuple of all the arguments after the required ones.

In [38]:
def log(message, *values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))

log('My numbers are', 1, 2)
log('Hi there')

My numbers are: 1, 2
Hi there


In [39]:
log('Favourite numbers', *[7, 9, 21])

Favourite numbers: 7, 9, 21


In [42]:
# Problem 1: variable arguments are always returned into a tuple
# exhausting generators, with a tuple with all the generators
# best to use it where you know the number of inputs 
# will be reasonably small
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)
    
my_func(*my_generator())

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


In [43]:
# Problem 2: can't add new positional arguments without migrating
def log(sequence, message, *values):
    if not values:
        print('%s: %s' % (sequence, message))
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s: %s' % (sequence, message, values_str))
        
log(1, 'Favourites', 7, 33) # new usage fine
log('Favourites', 7, 33) # old usage breaks

1: Favourites: 7, 33
Favourites: 7: 33


## Item 19: Provide optional behaviour with keyword arguments
Python can use both positonal and keyword arguments.

In [1]:
def remainder(number, divisor):
    return number % divisor

assert remainder(20, 7) == 6

All positional arguments can also be passed by keyword, in any order, as long as all the required positional arguments are specified.

In [2]:
remainder(20, 7)
remainder(20, divisor=7)
remainder(number=20, divisor=7)
remainder(divisor=7, number=20)

6

Positional arguments need to be specified before keyword ones.

In [3]:
remainder(number=20, 7)

SyntaxError: positional argument follows keyword argument (<ipython-input-3-9265fd4030d2>, line 1)

Each argument can only be specified once.

In [4]:
remainder(20, number=7)

TypeError: remainder() got multiple values for argument 'number'

Advantages:
- make the function call clearer
- they can have default values specified in the function definition
- provide a powerful way to extend funtion's parameters while remaining backwards compatible with existing callers

In [5]:
# provide additional capabilities with default values
def flow_rate(weight_diff, time_diff):
    return weight_diff / time_diff

weight_diff = 0.5
time_diff = 3
flow = flow_rate(weight_diff, time_diff)
print('%.3f per second' % flow)

0.167 per second


In [7]:
def flow_rate(weight_diff, time_diff, period):
    return (weight_diff / time_diff) * period

flow_per_second = flow_rate(weight_diff, time_diff, 1)
print(flow_per_second)

0.16666666666666666


In [9]:
def flow_rate(weight_diff, time_diff, period=1):
    return (weight_diff / time_diff) * period

flow_per_second = flow_rate(weight_diff, time_diff)
flow_per_hour = flow_rate(weight_diff, time_diff, period=3600)

print(flow_per_second)
print(flow_per_hour)

0.16666666666666666
600.0


In [40]:
# backwards compatibility
def flow_rate(weight_diff, time_diff, period=1, units_per_kg=1):
    return ((weight_diff * units_per_kg) / time_diff) * period

pounds_per_hour = flow_rate(weight_diff, time_diff,
                            period=3600, units_per_kg=2.2)

print(pounds_per_hour)

1320.0


In [41]:
# optional arguments can still be specified as positional
# which can be confusing - always pass them by keyword
pounds_per_hour = flow_rate(weight_diff, time_diff, 3600, 2.2)
print(pounds_per_hour)

1320.0
